# Задача 1 (из файла с проверками гипотез)

Цель: Определить наиболее характерные режимы с помощью предобученной линейной регрессии.

Задачи:

1. Оценить принципиальную возможность обнаруживать режимы таким способом

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from utils import TARGET_FEATURE, FIGSIZE, DATAFILE_NAMES, RANDOM_SEED, load_all, train_split, target_split
from scoring import clusterization_score

dfs = load_all()
for df in dfs:
    model = AgglomerativeClustering(len(df[TARGET_FEATURE].dropna().unique()))
    model.fit(*target_split(df.dropna()))
    break

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator
from sklearn.metrics import confusion_matrix

In [ ]:
from utils import TARGET_FEATURE, FIGSIZE, DATAFILE_NAMES, RANDOM_SEED, load_all, train_split, target_split
from features import f_derivative, t_min_max, t_q_q, t_stationary, t_dropna, t_limit

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
dfs = load_all()
funs = [f_derivative, t_dropna, t_q_q, t_dropna, t_limit]
funs = [t_q_q, t_dropna, t_limit]
funs = [t_min_max, t_dropna]
for fun in funs:
    dfs = [fun(df) for df in dfs]

In [ ]:
def get_data_for_test(df, mode, window):
    t2 = (df[TARGET_FEATURE] == mode).rolling(window).apply(all).idxmax()
    t1 = df.index[np.argmax(df.index == t2) - window + 1]
    return target_split(df[(df.index > t1) & (df.index < t2)])

## Оценка возможности реализации идеи

Задачи:

1. Обучить линейную регрессию на малой части данных, задав в качестве таргета любую константу
1. Посмотреть как она себя будет вести на всем временном диапазоне.

In [ ]:
class TestModel(BaseEstimator):
    def __init__(self, fit_type=2):
        self.model = LinearRegression()
        self.fit_type = fit_type
    
    def fit(self, x, y):
        if self.fit_type == 1:
            # 1 try to fit linear regression by lsa
            self.model.coef_ = np.linalg.lstsq(x.values, y.values, -1)[0]
            self.model.intercept_ = 0
        elif self.fit_type == 2:
            # 2 try to set random coeffs
            self.model.coef_ = (x.iloc[0] * 0 + 1 / x.shape[1]).values
            self.model.intercept_ = 0
        elif self.tfit_typeype == 3:
            # 3 try to fit lr model
            self.model.fit(x, y)
        else:
            raise ValueError(f"unknown fit_type value {self.fit_type}")
        return self
    
    def predict(self, x, *args, how='series'):
        y = self.model.predict(x, *args)
        if how == 'series':
            return pd.Series(y, index=x.index)
        return y

In [ ]:
mode = 1
data = [(get_data_for_test(df, mode, 200), target_split(df)) for df in dfs]
models = [TestModel().fit(*dff[0]) for dff in data]

In [ ]:
def plot_violin(res, t1):
    _, ax = plt.subplots(figsize=FIGSIZE)
    sns.violinplot(t1.fillna(-1), res, scale='width', inner='box', ax=ax)
    plt.show()

for model, ((df0, t0), (df1, t1)) in zip(models, data):
    plot_violin(model.predict(df1), t1)

Нужно посмотреть из каких режимов в какие возможен переход и есть ли там различия по расчетной метрике.

**Вывод**: для грубой оценки идея работает.

## Грубая попытка реализации

Задачи:

1. Подготовить модель
1. Определить режимы
1. Оценить точность


**Проверка не проведена из-за очевидной несостоятельности идеи. Линейная комбинация не позволяет эффективно отделить одни режимы от других (см. рисунки выше), поэтому она может быть использована как отдельный признак, но не как киллерфича**

In [ ]:
# # get model from previous cells
# model = model

# # predict over df
# res = model.predict(df1)

# _, ax = plt.subplots(figsize=FIGSIZE)
# res.plot(ax=ax)


# _, ax = plt.subplots(figsize=FIGSIZE)
# sns.scatterplot(df1.index, res, hue=t1.fillna(-1), s=0.5, ax=ax)